In [2]:
# 사전 download 받아야하는 module
!pip install sentence_transformers
!pip install konlpy
!pip install transformers
!pip install sentencepiece
!pip install sacremoses

In [3]:
import pandas as pd
import numpy as np
import itertools

# KeyBert 관련 import
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 감정분석을 위한 impmort
from tqdm import tqdm
import os
import re
from transformers import *
import tensorflow as tf
import json

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 키워드 추출

## 1. 전처리

In [39]:
# 크롤링한 book data 불러오기

data = pd.read_csv('/content/drive/MyDrive/code_states/cp1/book_table.csv', encoding = 'UTF-8')

In [40]:
data.head()

,isbn,item_id,category,title,author,publisher,p_date,price,page,avg_score,description
0,9791191825435,294902251,가정/요리/뷰티,맛있어서 지속 가능한 디디미니 다이어트 레시피,미니 박지우 지음,빅피시,2022-05-23,16920,296,10.0,매년 수만 개의 감량 후기가 잇따르는 고단백 요리로 수많은 다이어터의 찬사를 받고 ...
1,9791192107608,285827577,가정/요리/뷰티,열두 달 향신료 카레,김민지 지음,세미콜론,2022-06-20,18000,224,0.0,커다란 간판 없이도 사람들이 모여드는 서울 최고의 카레 성지. 오직 ‘향신료 카레’...
2,9791164261840,295216320,가정/요리/뷰티,페이스트리 테이블,박성채 지음,더테이블,2022-05-30,28800,256,10.0,디저트 카페 창업을 위한 레시피 & 컨설팅 북. 디저트 카페를 운영하는 데 있어 가...
3,9791138323284,296057122,가정/요리/뷰티,소워니놀이터의 말랑말랑 스퀴시,조윤성 지음,시대인,2022-07-05,18000,194,0.0,기존의 종이놀이를 뛰어넘어 도안과 도안 사이에 솜을 넣어 말랑말랑하고 폭신폭신한 촉...
4,9791140700233,296633531,가정/요리/뷰티,맛있는 게 좋아서 양조절 다이어트,한아름 지음,길벗,2022-06-22,16650,228,9.8,"매일 습관만 바꿔도 15kg이 빠진다. 고기도 탄수화물도 디저트도 포기할 수 없는,..."


In [41]:
len(data)

3090

In [42]:
# 결측치 제거해주기

data = data.dropna(axis=0)

In [43]:
len(data)

2955

In [44]:
# 전처리

drop_data = data[~data['author'].str.contains('지음')]
drop_data = drop_data[~drop_data['author'].str.contains('엮음')]
drop_data = drop_data[~drop_data['author'].str.contains('글')]
drop_data = drop_data[~drop_data['author'].str.contains('그림')]
drop_data = drop_data[~drop_data['author'].str.contains('저자')]
drop_data = drop_data[~drop_data['author'].str.contains('원작')]
data.drop(index=drop_data.index, inplace=True)
data.drop(index=data[data['category']=='전집/중고전집'].index, inplace=True)
def writer(str):
    words = ['엮음', '글', '그림', '저자', '원작']
    for word in words:
        str = str.replace(word, '지음')
    str = str.split('지음')[0].strip()
    str = str.split('외')[0].strip()
    str = re.sub(r'[A-Z]+.', '', str)
    str = str.replace('.', ', ')
    str = str.replace('(', ', ')
    str = str.replace(')', ', ')
    return str
data['author'] = data['author'].apply(lambda x: writer(x))
data.drop(index=data[data['author']==''].index, inplace=True) 

In [45]:
len(data)

2916

In [46]:
import re

def del_re(doc):
    myStr = re.sub('[^ㄱ-힗]', '', doc)
    return myStr

for i in tqdm(range(1)):
    data['description'] = data['description'].apply(del_re)

100%|██████████| 1/1 [00:00<00:00, 13.06it/s]


In [47]:
len(data)

2916

In [50]:
# 책의 의미를 담지 못하는 description 제외

def chk_doc(doc):
    if len(doc) < 53:
        return 1
    else :
        return 0

In [51]:
data['chk'] = data['description'].apply(chk_doc)

In [52]:
mask_2 = (data['chk']==1)
data = data[~mask_2]

In [53]:
len(data)

2802

In [54]:
data = data.reset_index(drop=True)
data

,isbn,item_id,category,title,author,publisher,p_date,price,page,avg_score,description,chk
0,9791191825435,294902251,가정/요리/뷰티,맛있어서 지속 가능한 디디미니 다이어트 레시피,미니 박지우,빅피시,2022-05-23,16920,296,10.0,매년수만개의감량후기가잇따르는고단백요리로수많은다이어터의찬사를받고있는디디미니가올해도어김...,0
1,9791192107608,285827577,가정/요리/뷰티,열두 달 향신료 카레,김민지,세미콜론,2022-06-20,18000,224,0.0,커다란간판없이도사람들이모여드는서울최고의카레성지오직향신료카레하나만을먹기위해웨이팅도불사...,0
2,9791164261840,295216320,가정/요리/뷰티,페이스트리 테이블,박성채,더테이블,2022-05-30,28800,256,10.0,디저트카페창업을위한레시피컨설팅북디저트카페를운영하는데있어가장기본이되는핵심메뉴그리고크림...,0
3,9791138323284,296057122,가정/요리/뷰티,소워니놀이터의 말랑말랑 스퀴시,조윤성,시대인,2022-07-05,18000,194,0.0,기존의종이놀이를뛰어넘어도안과도안사이에솜을넣어말랑말랑하고폭신폭신한촉감의스퀴시를만들수있...,0
4,9791140700233,296633531,가정/요리/뷰티,맛있는 게 좋아서 양조절 다이어트,한아름,길벗,2022-06-22,16650,228,9.8,매일습관만바꿔도이빠진다고기도탄수화물도디저트도포기할수없는맛있는게너무좋은다이어터들에게딱...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2797,9788952227829,35029230,로맨스소설,미 비포 유,조조 모예스,살림,2013-12-24,13500,536,8.9,영국에서입소문만으로최고의베스트셀러가되고이후출간된독일에서는밀리언셀러로서엄청난판매량을기...,0
2798,9788952766441,18593792,로맨스소설,그레이의 50가지 그림자 1,제임스,시공사,2012-08-08,10800,416,6.8,년월미국순문학출판사인랜덤하우스빈티지는그레이의가지그림자가출간석달만에시리즈포함총천백만부...,0
2799,9791189604110,244280066,로맨스소설,상견니,지엔치펑,리플레이,2020-07-14,14220,432,9.8,년대만최고의화제작드라마소설죽은남자친구왕취안성에대한그리움을떨치지못한황위쉬안은이세계의또...,0
2800,9791135475832,290418968,로맨스소설,나쁜 생각이 들어서,요안나,로코코,2022-03-17,12150,560,9.9,요안나장편소설졸지에거액의빚과생면부지의어린이복동생이생긴서희학교선배이자금융그룹의대표서지...,0


## 2. Bert 모델 적용

In [ ]:
# 문서로부터 추출한 키워드들을 SBERT를 통해 수치화
# 한국어를 포함하고 있는 다국어 SBERT load

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

사이킷런의 CountVectorizer를 사용하여 단어 추출
- n_gram_range의 인자를 사용하여 쉽게 n-gram 추출 가능
- ex) (2, 3)으로 설정하면 결과 후버는 2개의 단어 또는 3개의 단어를 한 묶음으로 간주하여 추출
- 이번 프로젝트에서는 단어 하나의 키워드 추출을 위해 (1, 1)로 설정

키워드 추출 후, 각 단어를 임베딩 벡터화(수치화) 시켜준다.
- 해당 작업은, SBERT모델에서 각 단어의 벡터를 호출해온다.

In [56]:
def keyword_out(doc):

    # okt 객체 생성
    okt = Okt()

    # 태깅 작업 후, 품사가 명사인 단어들을 모두 join 해준다.
    tokenized_doc = okt.pos(doc)
    tokenized_nouns = ' '.join(okt.nouns(doc))

    # 최소 1, 최대 1 길이의 단어를 추출한다.
    n_gram_range = (1, 1)

    try:
        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names_out()
    except:
        return 0

    # global model
    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)

    # top_n : 추출할 키워드 갯수
    # nr_candidates : 상위 n개의 키워드를 선택하고 이들 중 가장 유사성이 낮은 5개 선택
    top_n = 5
    nr_candidates = 10

    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim
        # else:
        #     return '키워드 불가'

    return [words_vals[idx] for idx in candidate]

In [59]:
data['keyword'] = -1

In [ ]:
# 키워드 추출 후 keyword 컬럼에 저장

for i in tqdm(range(len(data))):
    data['keyword'][i] = keyword_out(data['description'][i])

  0%|          | 0/2802 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 99%|█████████▉| 2778/2802 [3:28:41<01:27,  3.64s/it]

In [61]:
data

,isbn,item_id,category,title,author,publisher,p_date,price,page,avg_score,description,chk,keyword
0,9791191825435,294902251,가정/요리/뷰티,맛있어서 지속 가능한 디디미니 다이어트 레시피,미니 박지우,빅피시,2022-05-23,16920,296,10.0,매년수만개의감량후기가잇따르는고단백요리로수많은다이어터의찬사를받고있는디디미니가올해도어김...,0,"[활용, 요리, 찬사, 디저트, 다이어트]"
1,9791192107608,285827577,가정/요리/뷰티,열두 달 향신료 카레,김민지,세미콜론,2022-06-20,18000,224,0.0,커다란간판없이도사람들이모여드는서울최고의카레성지오직향신료카레하나만을먹기위해웨이팅도불사...,0,"[노란색, 먹기, 열두달, 향신료, 서울]"
2,9791164261840,295216320,가정/요리/뷰티,페이스트리 테이블,박성채,더테이블,2022-05-30,28800,256,10.0,디저트카페창업을위한레시피컨설팅북디저트카페를운영하는데있어가장기본이되는핵심메뉴그리고크림...,0,"[시뮬레이션, 파티, 창업, 메뉴, 디저트]"
3,9791138323284,296057122,가정/요리/뷰티,소워니놀이터의 말랑말랑 스퀴시,조윤성,시대인,2022-07-05,18000,194,0.0,기존의종이놀이를뛰어넘어도안과도안사이에솜을넣어말랑말랑하고폭신폭신한촉감의스퀴시를만들수있...,0,"[손재주, 칭찬, 놀이, 놀이터, 만족감]"
4,9791140700233,296633531,가정/요리/뷰티,맛있는 게 좋아서 양조절 다이어트,한아름,길벗,2022-06-22,16650,228,9.8,매일습관만바꿔도이빠진다고기도탄수화물도디저트도포기할수없는맛있는게너무좋은다이어터들에게딱...,0,"[만천, 습관, 탄수화물, 디저트, 다이어트]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,9788952227829,35029230,로맨스소설,미 비포 유,조조 모예스,살림,2013-12-24,13500,536,8.9,영국에서입소문만으로최고의베스트셀러가되고이후출간된독일에서는밀리언셀러로서엄청난판매량을기...,0,"[첫출근, 경제, 남녀, 유머, 독일]"
2798,9788952766441,18593792,로맨스소설,그레이의 50가지 그림자 1,제임스,시공사,2012-08-08,10800,416,6.8,년월미국순문학출판사인랜덤하우스빈티지는그레이의가지그림자가출간석달만에시리즈포함총천백만부...,0,"[판매, 뉴욕타임스, 아마존닷컴, 로맨스소설, 억만장자]"
2799,9791189604110,244280066,로맨스소설,상견니,지엔치펑,리플레이,2020-07-14,14220,432,9.8,년대만최고의화제작드라마소설죽은남자친구왕취안성에대한그리움을떨치지못한황위쉬안은이세계의또...,0,"[의혹, 확신, 영혼, 소설, 그리움]"
2800,9791135475832,290418968,로맨스소설,나쁜 생각이 들어서,요안나,로코코,2022-03-17,12150,560,9.9,요안나장편소설졸지에거액의빚과생면부지의어린이복동생이생긴서희학교선배이자금융그룹의대표서지...,0,"[어린이, 거짓말, 학교, 장편소설, 금융]"


In [62]:
# data 추출

data.to_csv('/content/drive/MyDrive/code_states/cp1/book_table_after.csv')

# 감성분석

한국어 단발성 대화 데이터셋 파일을 가져와 BERT 모양에 사전학습을 시켜주고, </br>
크롤링한 데이터를 파인튜닝하여 프로젝트의 방향에 맞는 출력 labeling을 진행한다.

## 1. 전처리

In [134]:
import pandas as pd
emo = pd.read_excel('/content/drive/MyDrive/code_states/cp1/한국어_단발성_대화_데이터셋.xlsx')

In [135]:
emo

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [136]:
emo.loc[(emo['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
emo.loc[(emo['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
emo.loc[(emo['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
emo.loc[(emo['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
emo.loc[(emo['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
emo.loc[(emo['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
emo.loc[(emo['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

data_list = []
for q, label in zip(emo['Emotion'], emo['Sentence'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [137]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])

[0, '언니 동생으로 부르는게 맞는 일인가요..??']
[1, '기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?']
[2, '당연히 그렇게 해야지 우리나라도 판매를 중단하라']


In [138]:
df_emo = pd.DataFrame(data_list, columns=['category', 'data'])

In [139]:
df_emo

,category,data
0,0,언니 동생으로 부르는게 맞는 일인가요..??
1,0,그냥 내 느낌일뿐겠지?
2,0,아직너무초기라서 그런거죠?
3,0,유치원버스 사고 낫다던데
4,0,근데 원래이런거맞나요
...,...,...
38589,6,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..
38590,6,재미가 없으니 망하지
38591,6,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...
38592,6,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ


In [140]:
print(len(df_emo))

38594


In [141]:
df_emo['data'] = df_emo['data'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
df_emo['data'] = df_emo['data'].str.replace(r'\t+', " ", regex=True)
df_emo['data'] = df_emo['data'].str.replace(r'[\\n]+'," ", regex=True)

In [142]:
from sklearn.model_selection import train_test_split
                                                         
train_df, test_df = train_test_split(df_emo, test_size=0.25, random_state=42)

In [143]:
train_df = train_df.reset_index().drop(['index'], axis=1).reset_index()

In [144]:
train_df.head()

,index,category,data
0,0,3,휴 요세 몇일간
1,1,5,백주부님딸바보 되시겠네요
2,2,1,헐 게임에서만 보던 오간도;;
3,3,3,저 지팡이 아직 못샀는데ㅠㅠㅠ RIP
4,4,4,그냥 그럼 축구장을 하나 더 사


In [145]:
train_df['category'].value_counts()

5    4534
1    4414
2    4245
0    4090
6    4029
3    3995
4    3638
Name: category, dtype: int64

In [146]:
len(train_df)

28945

### Test data 가져오기

In [177]:
review = pd.read_csv('/content/drive/MyDrive/code_states/cp1/review_table.csv')

In [178]:
review.head()

,isbn,user_id,score,rev_content
0,9791191825435,gkxmzbsl,10,"이 전에 3,4권도 요리의 ‘요‘자도 알지못하던 저한테 있어서 빛과 같은 희망을 준..."
1,9791191825435,닉닉,10,"이전 책들도 구입했는데, 집밥 먹을때는 거의 항상 디디미니님 레시피 따라하게 되더라..."
2,9791191825435,zin,10,드디어 나왔나요!!!바로 주문합니다!!! 책 잘 보고 다이어트 도전 해볼께요!!! ...
3,9791191825435,이쁜카라얌,10,요 앞 버젼 사려다가 최신간으로 구매 했어요다이어트를 해도 점점 찌는 나 드디어 굴...
4,9791191825435,feelingood,10,디디미니 언니 레시피 언제나 믿고 만들어먹어요~ 이번 책엔 또 어떤 맛있는 레시피가...


In [179]:
review = review.dropna(axis=0)

In [180]:
review = review.reset_index()

In [181]:
review

,index,isbn,user_id,score,rev_content
0,0,9791191825435,gkxmzbsl,10,"이 전에 3,4권도 요리의 ‘요‘자도 알지못하던 저한테 있어서 빛과 같은 희망을 준..."
1,1,9791191825435,닉닉,10,"이전 책들도 구입했는데, 집밥 먹을때는 거의 항상 디디미니님 레시피 따라하게 되더라..."
2,2,9791191825435,zin,10,드디어 나왔나요!!!바로 주문합니다!!! 책 잘 보고 다이어트 도전 해볼께요!!! ...
3,3,9791191825435,이쁜카라얌,10,요 앞 버젼 사려다가 최신간으로 구매 했어요다이어트를 해도 점점 찌는 나 드디어 굴...
4,4,9791191825435,feelingood,10,디디미니 언니 레시피 언제나 믿고 만들어먹어요~ 이번 책엔 또 어떤 맛있는 레시피가...
...,...,...,...,...,...
16318,17192,9791186561096,gksmfqor12,10,여행을 하면 남는건 사진밖에 없다고..
16319,17193,9791186561096,반가워요,10,저도 떠나보고 싶게끔 만드네요 ^^
16320,17194,9791186561096,Polnareff,10,20대만의 재기발랄함이 멋지네요
16321,17195,9791186561096,0ijimin0i,10,유럽 여행을 해보지는 못했지만 이책으로 인해 유럽에 대해더 자세히 알 수 있어서 너...


In [182]:
test_list = []
for q in review['rev_content']:
    data = []
    data.append(q)

    test_list.append(data)

In [183]:
test_set = pd.DataFrame(test_list, columns=['data'])
test_set = test_set.reset_index().drop(['index'], axis=1).reset_index()

In [184]:
test_set['data'] = test_set['data'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
test_set['data'] = test_set['data'].str.replace(r'\t+', " ", regex=True)
test_set['data'] = test_set['data'].str.replace(r'[\\n]+'," ", regex=True)

In [154]:
test_set.head()

,index,data
0,0,이 전에 3 4권도 요리의 요 자도 알지못하던 저한테 있어서 빛과 같은 희망을 준...
1,1,이전 책들도 구입했는데 집밥 먹을때는 거의 항상 디디미니님 레시피 따라하게 되더라...
2,2,드디어 나왔나요 바로 주문합니다 책 잘 보고 다이어트 도전 해볼께요 지긋지긋한 ...
3,3,요 앞 버젼 사려다가 최신간으로 구매 했어요다이어트를 해도 점점 찌는 나 드디어 굴...
4,4,디디미니 언니 레시피 언제나 믿고 만들어먹어요 이번 책엔 또 어떤 맛있는 레시피가...


## 2. 사용할 Bert 모델 구축
- 이미 학습되어 있는 Bert모형에 train data를 학습시켜준다.
    - 이를 위해, train data를 bert 모형 input에 맞게 바꾸어준다.
- train data가 적용된 bert 모델의 마지막 부분에 목적에 맞는 output layer를 정의한다.
    - 감정 label이 모두 7개이므로, `softmax` 함수를 사용하고, 7개의 노드로 구현한다.

In [ ]:
# bert input 만들기

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [156]:
# train data를 bert model input 에 맞게 변환해주는 함수

def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 512 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df)), position=0, leave=True):
        # token : 문장을 토큰화함
        # 우리가 입력할 데이터를 토큰화 한다.
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation = True, padding='max_length')
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        # 최대 길이의 문장에 맞춰 단어가 없는 문장은 0으로 패딩처리
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        # 만약, 문장이 대화 형식이었으면 문장의 전후관계를 나타내주는 세그먼트의 구분이 필요
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(감정 인덱싱)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 512
BATCH_SIZE = 20
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "data"
# 긍정인지 부정인지를 (감정 인덱싱) 포함하고 있는 칼럼
LABEL_COLUMN = "category"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train_df)

100%|██████████| 28945/28945 [00:10<00:00, 2802.07it/s]


In [157]:
# TPU 객체 만들기
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.110.32.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.110.32.242:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [158]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [7]:
# 데이터에 맞는 bertmodel 정의
def create_sentiment_bert():

  # 버트 pretrained 모델 로드
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True)

  # 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')

  # 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

  bert_outputs = bert_outputs[1]
  sentiment_first = tf.keras.layers.Dense(7, activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02))(bert_outputs)
  sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

  # 옵티마이저는 간단하게 Adam 옵티마이저 활용
  sentiment_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  
  return sentiment_model

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
with strategy.scope():
  
  sentiment_model = create_sentiment_bert()
  sentiment_model.fit(train_x, train_y, epochs=3, shuffle=False, batch_size=20)
  sentiment_model.save_weights(os.path.join('/content/drive/MyDrive/code_states/cp1',"sentiment_model.h5"))

In [186]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]
SEQ_LEN = 512
DATA_COLUMN = 'data'
# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

In [187]:
test_set = predict_load_data(test_set)

100%|██████████| 16323/16323 [00:11<00:00, 1362.85it/s]


In [ ]:
test_set

[array([[  101,  9638, 91069, ...,     0,     0,     0],
        [  101, 84253,  9739, ...,     0,     0,     0],
        [  101,  9113, 48446, ...,     0,     0,     0],
        ...,
        [  101,  8996, 96404, ...,     0,     0,     0],
        [  101, 66324, 11287, ...,     0,     0,     0],
        [  101,  9420, 66540, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [164]:
path = "/content/drive/MyDrive/code_states/cp1"

import shutil
if "bert" not in os.listdir():
  os.makedirs("bert")
else:
  pass

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree(path, "bert")

In [ ]:
import sys
mod = sys.modules[__name__]
# TPU를 활용하기 위해 context로 묶어주기
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
  sentiment_model = create_sentiment_bert()
  
  sentiment_model.load_weights(os.path.join("bert", "sentiment_model.h5"))
  setattr(mod, 'model', sentiment_model)
  setattr(mod, 'pred_', sentiment_model.predict(test_set, batch_size=1))

In [13]:
# 정의한 모델 summary

sentiment_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 512)]        0           []                               
                                                                                                  
 input_segment (InputLayer)     [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model_3 (TFBertModel)  TFBaseModelOutputWi  177853440   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]',      

In [166]:
# 분석 모델을 거쳐 나온 값들을
# labeling에 사용하여
# 각 label을 부여시켜주는 함수

def mean_answer_label(*preds):
  preds_sum = np.zeros(preds[0].shape[0])
  for pred in preds:
    preds_sum += np.argmax(pred, axis=-1)
  return np.round(preds_sum/len(preds), 0).astype(int)

In [189]:
# test sentence를 bert model에 입력시킬 수 있는
# 입력 형태로 변환시켜주는 함수
def sentence_convert_data(data):
    global tokenizer
    
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, padding='max_length', truncation=True)
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]



# 감성 분석 모델을 통해 label predict를 진행하는 함수
def category_evaluation_predict(sentence):

    global mod
    data_x = sentence_convert_data(sentence)
    
    global strategy
    with strategy.scope():
      
      setattr(mod, 'emo', model.predict(data_x, batch_size=1))

    

    preds = str(mean_answer_label(emo).item())
    
    return int(preds)

In [190]:
# 감성 label을 기입할 category 컬럼 추가

review['category'] = -1

In [191]:
# 사전학습 및 파인튜닝한 Bert 모델로 test data labeling 진행

with strategy.scope():
    for i in tqdm(range(len(review))):
        review['category'][i] = category_evaluation_predict(review['rev_content'][i])

  0%|          | 0/16323 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 16323/16323 [3:30:19<00:00,  1.29it/s]


In [192]:
review

,index,isbn,user_id,score,rev_content,category
0,0,9791191825435,gkxmzbsl,10,"이 전에 3,4권도 요리의 ‘요‘자도 알지못하던 저한테 있어서 빛과 같은 희망을 준...",0
1,1,9791191825435,닉닉,10,"이전 책들도 구입했는데, 집밥 먹을때는 거의 항상 디디미니님 레시피 따라하게 되더라...",5
2,2,9791191825435,zin,10,드디어 나왔나요!!!바로 주문합니다!!! 책 잘 보고 다이어트 도전 해볼께요!!! ...,0
3,3,9791191825435,이쁜카라얌,10,요 앞 버젼 사려다가 최신간으로 구매 했어요다이어트를 해도 점점 찌는 나 드디어 굴...,0
4,4,9791191825435,feelingood,10,디디미니 언니 레시피 언제나 믿고 만들어먹어요~ 이번 책엔 또 어떤 맛있는 레시피가...,5
...,...,...,...,...,...,...
16318,17192,9791186561096,gksmfqor12,10,여행을 하면 남는건 사진밖에 없다고..,0
16319,17193,9791186561096,반가워요,10,저도 떠나보고 싶게끔 만드네요 ^^,3
16320,17194,9791186561096,Polnareff,10,20대만의 재기발랄함이 멋지네요,5
16321,17195,9791186561096,0ijimin0i,10,유럽 여행을 해보지는 못했지만 이책으로 인해 유럽에 대해더 자세히 알 수 있어서 너...,5


In [196]:
# label 분포 확인

review['category'].value_counts()

5    6437
0    3305
4    2731
3    2414
1    1045
6     270
2     121
Name: category, dtype: int64

In [195]:
# 감성 분석을 통해 review 감성을 분류한 category 컬럼 추가된 csv 파일 추출

review.to_csv('/content/drive/MyDrive/code_states/cp1/review_table_after.csv')